## Importer le modèle

In [ ]:
from keras.models import load_model
import numpy as np
model = load_model('GoodModel/model_multi_Recommandation6.h5')

model.summary()

## Importer l'image

In [ ]:
import rasterio
import rasterio
#Mettre les chemins d'accès aux bandes utilisées par l'entrainement du
#modèle utilisé dans le bon ordre dans le tableau ci-dessous.
bands = ['gauche/fall/T48PYU_20191025T030811_B08_10m.jp2',
        'gauche/fall/T48PYU_20191025T030811_B03_10m.jp2',
        'gauche/winter/T48PYU_20191224T031131_B08_10m.jp2',
        'gauche/winter/T48PYU_20191224T031131_B03_10m.jp2',
        'gauche/spring/T48PYU_20200318T030539_B08_10m.jp2',
        'gauche/spring/T48PYU_20200318T030539_B03_10m.jp2',
        ]

In [ ]:
IMAGE_PIXEL = 15
IMAGE_WIDTH = 732
BAND_WIDTH = len(bands)

In [ ]:
def reshape(image, width):
    """Transforme une image raster pour pouvoir être utilisable par le réseau de neurones"""
    reshaped_image = []
    for i in range(len(image[0])):
        reshaped_row = []
        for j in range(len(image[0][0])):
            reshaped_cell = []
            for k in range(width):
                reshaped_cell.append(image[k][i][j] / 65535)
            reshaped_row.append(reshaped_cell)
        reshaped_image.append(reshaped_row)
    return reshaped_image

In [ ]:
def split_band(band, width, pixel_length):
    """Split une bande de Sentinel-2 en bande plus petites images"""
    result = []
    my_band = band.read(1)
    for x in range(width):
        columns = []
        for y in range(width):
            lines = []
            for i in range(pixel_length):
                cells = []
                for j in range(pixel_length):
                    cells.append(my_band[i + IMAGE_PIXEL * x][j + IMAGE_PIXEL * y])
                lines.append(cells)
            columns.append(lines)
        result.append(columns)
    return result

In [ ]:
def predict(bands, model, output_path="output", output_name="prédiction_multiple"):
    """Crée l'image de sortie en rgb (r:café, b:poivre, g:autre)"""
    images = []
    width = None
    height = None
    crs = None
    transform = None
    dtype = None
    for band in bands:
        image = rasterio.open(band, driver='JP2OpenJPEG')
        images.append(split_band(image, IMAGE_WIDTH, IMAGE_PIXEL))
        if width == None:
            width = image.width
            height = image.height
            crs = image.crs
            transform = image.transform
            dtype = image.dtypes[0]
        image.close()
        print("Fin", band)
    prediction_coffee = []
    prediction_pepper = []
    prediction_other = []
    for x in range(IMAGE_WIDTH):
        row_prediction_coffee = []
        row_prediction_pepper = []
        row_prediction_other = []
        for y in range(IMAGE_WIDTH):
            smaller_image = []
            for image in images:
                smaller_image.append(image[x][y])
            value = model.predict(np.expand_dims(reshape(smaller_image, BAND_WIDTH), axis=0))
            for i in range(IMAGE_PIXEL):
                row_prediction_other.append(value[0][0] * 65535)
                row_prediction_pepper.append(value[0][1] * 65535)
                row_prediction_coffee.append(value[0][2] * 65535)
        for i in range(IMAGE_PIXEL):
            prediction_coffee.append(np.array(row_prediction_coffee).astype('uint16'))
            prediction_pepper.append(np.array(row_prediction_pepper).astype('uint16'))
            prediction_other.append(np.array(row_prediction_other).astype('uint16'))
    output = rasterio.open(output_path + '/' + output_name + '.tiff', 'w', driver='Gtiff', 
                          width=width, height=height, count=3, crs=crs, 
                          transform=transform, 
                          dtype=dtype)
    output.write(prediction_pepper, 3)
    output.write(prediction_other, 2)
    output.write(prediction_coffee, 1)
    output.close()
    print("Le fichier de prédiction a été créé.")

In [ ]:
predict(bands, model=model, output_path="output", output_name="multi_Recommandation6")